**Предсказание стоимости жилья**

В проекте нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучим модель и сделаем предсказания на тестовой выборке. Для оценки качества модели используем метрики RMSE, MAE и R2.

# Подготовка данных

In [1]:
import pandas as pd 
import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import (
    BinaryClassificationEvaluator, 
    MulticlassClassificationEvaluator,
    RegressionEvaluator
)
pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder    
elif int(pyspark_version[:1]) == 2:
    from pyspark.ml.feature import OneHotEncodeEstimator

In [2]:
RANDOM_SEED = 2022

spark = (
    SparkSession
    .builder
    .master("local")
    .appName("Learning :inearRegression")
    .getOrCreate() 
)

data = spark.read.option('header', 'true').csv('housing.csv', inferSchema = True)
data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [3]:
# все данные
housing = data.select(
    'longitude', 'latitude',
    'housing_median_age', 'total_rooms', 'total_bedrooms',
    'population', 'households', 'median_income', 
    'median_house_value', 'ocean_proximity'
    )

# только чсловые признаки
housing_no_cat = data.select(
    'longitude', 'latitude',
    'housing_median_age', 'total_rooms', 'total_bedrooms',
    'population', 'households', 'median_income', 
    'median_house_value'
    )

In [4]:
# обзор данных
housing.show(3)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
only showing top 3 rows



In [5]:
# базовые статистики
housing.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


In [6]:
# количество пропусков
(
    housing
    .select(
        [F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in housing.columns]
    )
    .toPandas()
)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,0,0,0,0,207,0,0,0,0,0


In [7]:
# заполним пропуски медианным значением
housing = (
    housing
    .fillna(
        {
            'total_bedrooms': housing.agg(F.mean('total_bedrooms'))
            .first()[0]
        }
    )
)

housing_no_cat = (
    housing_no_cat
    .fillna(
        {
            'total_bedrooms': housing.agg(F.mean('total_bedrooms'))
            .first()[0]
        }
    )
)

In [8]:
# три типа признаков, категориальные,числовые и целевой
categorical_cols = ['ocean_proximity']
numerical_cols  = [
    'longitude', 'latitude',
    'housing_median_age', 'total_rooms', 'total_bedrooms',
    'population', 'households', 'median_income'
]
target = "median_house_value" 

## Разделение на выборки и обработка признаков

In [9]:
# полный набор данных
train_data, test_data = housing.randomSplit([.8,.2], seed=RANDOM_SEED)

# без категорий
train_data_no_cat, test_data_no_cat = housing_no_cat.randomSplit([.8,.2], seed=RANDOM_SEED)

# трансформатор категориальных признаков
indexer = StringIndexer(
    inputCols=categorical_cols,
    outputCols=[c+'_idx' for c in categorical_cols],
    handleInvalid = 'keep'
)
model_indexer = indexer.fit(train_data)
train_data = model_indexer.transform(train_data)
test_data = model_indexer.transform(test_data)

# OHE-трансформатор категориальных признаков
encoder = OneHotEncoder(
    inputCols=[c+'_idx' for c in categorical_cols],
    outputCols=[c+'_ohe' for c in categorical_cols]
)
model_encoder = encoder.fit(train_data)
train_data = model_encoder.transform(train_data)
test_data = model_encoder.transform(test_data)

# объединение признаков в вектор
categorical_assembler = VectorAssembler(
    inputCols=[c+'_ohe' for c in categorical_cols],
    outputCol="categorical_features"
)
train_data = categorical_assembler.transform(train_data)
test_data = categorical_assembler.transform(test_data)

# шкалирование
numerical_assembler = VectorAssembler(
    inputCols=numerical_cols, 
    outputCol="numerical_features"
)
train_data = numerical_assembler.transform(train_data)
test_data = numerical_assembler.transform(test_data)
train_data_no_cat = numerical_assembler.transform(train_data_no_cat)
test_data_no_cat = numerical_assembler.transform(test_data_no_cat)

# нормализация
standardScaler = StandardScaler(
    inputCol='numerical_features',
    outputCol="numerical_features_scaled"
)
model_standardScaler = standardScaler.fit(train_data)
model_standardScaler_no_cat = standardScaler.fit(train_data_no_cat)
train_data = model_standardScaler.transform(train_data)
test_data = model_standardScaler.transform(test_data)
train_data_no_cat = model_standardScaler_no_cat.transform(train_data_no_cat)
test_data_no_cat = model_standardScaler_no_cat.transform(test_data_no_cat)

# объединение, полный набор данных
all_features = ['numerical_features_scaled', 'categorical_features']

final_assembler = VectorAssembler(inputCols=all_features, 
                                  outputCol="features") 
train_data = final_assembler.transform(train_data)
test_data = final_assembler.transform(test_data)

# объединение, без категорий
all_features_no_cat = [
    'longitude', 'latitude',
    'housing_median_age', 'total_rooms', 'total_bedrooms',
    'population', 'households', 'median_income',
    'numerical_features', 'numerical_features_scaled'
]
final_assembler_no_cat = VectorAssembler(inputCols=all_features_no_cat, 
                                  outputCol="features") 
train_data_no_cat = final_assembler_no_cat.transform(train_data_no_cat)
test_data_no_cat = final_assembler_no_cat.transform(test_data_no_cat)

# Обучение моделей

In [10]:
# алгоритм прнимает подготовленные данные, полный набор данных
lr = LinearRegression(labelCol = target, maxIter=10, regParam=0.3, elasticNetParam=0.8)
model = lr.fit(train_data) 

25/02/14 16:22:12 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
25/02/14 16:22:12 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [11]:
# алгоритм прнимает подготовленные данные, без категорий
lr_no_cat = LinearRegression(labelCol = target, maxIter=10, regParam=0.3, elasticNetParam=0.8)
model_no_cat = lr_no_cat.fit(train_data_no_cat) 

In [12]:
# предсказания, полный набор данных
predictions = model.transform(test_data)
predictedLabes = predictions.select("median_house_value", "prediction")
predictedLabes.show()

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          103600.0|  182285.104944975|
|           50800.0|231881.45708222373|
|           58100.0|157977.37314810825|
|           68400.0|160956.83919210604|
|           72200.0| 179611.2824642522|
|           67000.0|170196.84260551736|
|           81300.0| 169350.0857670986|
|           70500.0| 182127.7343615702|
|           60000.0|159733.80720700452|
|          109400.0|201953.63070480118|
|           74100.0|167995.44826235448|
|           74700.0| 187343.4740787606|
|           90000.0|227818.66910462163|
|          104200.0|218948.29801876575|
|           74100.0| 175243.6540852181|
|           67500.0|166560.11338318104|
|          103100.0| 70862.23743311397|
|           92500.0|177441.04949920077|
|          128100.0| 235084.6981139189|
|           99600.0|204457.54931465432|
+------------------+------------------+
only showing top 20 rows



In [13]:
# предсказания, без категорий
predictions_no_cat = model_no_cat.transform(test_data_no_cat)
predictedLabes_no_cat = predictions_no_cat.select("median_house_value", "prediction")
predictedLabes_no_cat.show()

+------------------+-------------------+
|median_house_value|         prediction|
+------------------+-------------------+
|          103600.0| 105633.99332956038|
|           50800.0| 182063.35213976027|
|           58100.0| 107114.32935573999|
|           68400.0|  85810.84353321418|
|           72200.0|  126737.1699739662|
|           67000.0|  118292.2261051475|
|           81300.0| 116939.24293019203|
|           70500.0|  133571.8331590509|
|           60000.0| 109978.60671096016|
|          109400.0| 122626.33297799248|
|           74100.0| 120570.46014583111|
|           74700.0|  139405.8700640686|
|           90000.0| 177803.06818344025|
|          104200.0|  168116.6700751367|
|           74100.0| 124037.98498514853|
|           67500.0| 110847.92131673079|
|          103100.0|-15578.461096462328|
|           92500.0| 138983.03798170667|
|          128100.0| 187470.78506444115|
|           99600.0| 155908.51102677733|
+------------------+-------------------+
only showing top

In [14]:
# оценка метрик, полный набор данных
evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction")

rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})

print(f"housing - RMSE: {rmse}, MAE: {mae}, R2: {r2}")

housing - RMSE: 68977.59966256263, MAE: 50076.277840938405, R2: 0.6485742713919607


In [15]:
# оценка метрик, без категорий
evaluator_no_cat = RegressionEvaluator(labelCol=target, predictionCol="prediction")

rmse_no_cat = evaluator.evaluate(predictions_no_cat, {evaluator.metricName: "rmse"})
mae_no_cat = evaluator.evaluate(predictions_no_cat, {evaluator.metricName: "mae"})
r2_no_cat = evaluator.evaluate(predictions_no_cat, {evaluator.metricName: "r2"})

print(f"housing_no_cat - RMSE: {rmse_no_cat}, MAE: {mae_no_cat}, R2: {r2_no_cat}")

housing_no_cat - RMSE: 69598.32234695918, MAE: 50819.236254485484, R2: 0.6422209212136882


In [16]:
spark.stop()

# Анализ результатов

По итогу данного исследоавния можно сделать вывод о следующем:

- метрика RMSE находится на уровне 13.8% от максималього значения медианной стоимости дома в жилом массиве, это достойный показатель. Разница между моделями незначительная
- метрика MAE находится на уровне 10% от максималього значения медианной стоимости дома в жилом массиве, это достойный показатель. Разница между моделями незначительная.
- метрика R2 у двух моделей объясняет около 64% вариации, в данном случае достаточно низкий показатель. 